In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


This code does not give the right value for nPz for a given event from `momentumFuncBoost.ipybn` (See cell with print commands: print(eArr[0]),...).

Code is similar to the one for minimizing loss function for the stationary W case. The possible problems can be either the "complicated" mass function or larger number of parameters. I am not sure yet what it is.

In [2]:
eArr = np.array([36.72802525,-2.40983846,-12.08513295,38.7402323])
wArr = np.array([0.0, 0.0, 8.0704536, 80.704536])

In [3]:
%%cpp -d   

void fcn(int &npar, double *gin, double &f, double *par, int iflag){
    double pTSq = par[0];
    double pTotSq = par[1];
    double ePz = par[2];
    double nPz = par[3];

    double wMass = TMath::Sqrt(2*pow(pTSq,2) + 2 * TMath::Sqrt(pTotSq) * TMath::Sqrt(pTSq + pow(nPz,2)) - 2 * ePz * nPz);

    f = pow(wMass - 80.3, 2);
};

double minimization_func(double *wArr, double *elArr){
    double ePx = elArr[0];
    double ePy = elArr[1];
    double ePz = elArr[2];
    
    double pTSqEl = pow(ePx,2) + pow(ePy,2);
    double pTotSqEl = pTSqEl + pow(ePz,2);  
    
    TMinuit *gMinuit = new TMinuit(4);
    gMinuit->SetFCN(fcn);
    gMinuit->DefineParameter(0, "pTotSqEl", pTSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(1, "pTSqEl", pTotSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(2, "ePz", ePz, 1., 0., 0.);
    gMinuit->DefineParameter(3, "nPz", 22., 0.05, 0., 0.);
    gMinuit->FixParameter(0);
    gMinuit->FixParameter(1);
    gMinuit->FixParameter(2);
    gMinuit->Command("MIGRAD");

    double nPz, nPzErr;
    gMinuit->GetParameter(3, nPz, nPzErr);
    return nPz;

};

In [4]:
r.minimization_func(wArr,eArr)

-12.085132958419432

 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 pTotSqEl     1.35476e+03  1.00000e+00     no limits
     2 pTSqEl       1.50081e+03  1.00000e+00     no limits
     3 ePz         -1.20851e+01  1.00000e+00     no limits
     4 nPz          2.20000e+01  5.00000e-02     no limits
 **********
 **    1 **FIX           1
 **********
 **********
 **    2 **FIX           2
 **********
 **********
 **    3 **FIX           3
 **********
 **********
 **    4 **MIGRAD 
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-04
 FCN=3.3717e+06 FROM MIGRAD    STATUS=INITIATE        4 CALLS           5 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  pTotSqEl     1.35476e+03     fixed    